In [2]:

# 🚀 AI Tutor - Clean Working DataSci GPU Tutor (RAG + LLM + Game Mode)
import os
import pickle
import faiss
import torch
import gradio as gr
import numpy as np
import random
import requests
import ast
import re
import difflib
import json
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import requests
import random
import difflib
import re
from types import SimpleNamespace
import ast
import time
import cProfile
import io
import pstats
import contextlib
import sys
import traceback
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# Includes: FAISS RAG system, fallback to Falcon-7B-Instruct, and full game mode


# --- CONFIG ---
# EMBED_MODEL = "/scratch/ntiwar12/huggingfacesentence-transformers/all-MiniLM-L6-v2"
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
#"BAAI/bge-m3"
LLM_MODEL = "NousResearch/Hermes-2-Pro-Mistral-7B"
#"deepseek-ai/deepseek-coder-6.7b-instruct"
#"microsoft/phi-4"

CHUNK_FILE = "chunks.pkl"
INDEX_FILE = "faiss.index" 
URLS_FILE = "custom_urls.txt"

# --- Device Detection and Setup ---
def detect_and_setup_device():
    """Detect available device and configure models accordingly"""
    if torch.cuda.is_available():
        gpu_count = torch.cuda.device_count()
        gpu_name = torch.cuda.get_device_name(0) if gpu_count > 0 else "Unknown GPU"
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3) if gpu_count > 0 else 0
        
        print(f"🚀 GPU DETECTED: {gpu_name}")
        print(f"📊 GPU Memory: {gpu_memory:.1f} GB")
        print(f"🔢 GPU Count: {gpu_count}")
        
        try:
            # Test GPU availability with a small tensor operation
            test_tensor = torch.randn(10, 10).cuda()
            _ = test_tensor @ test_tensor
            print("✅ GPU is available and functional")
            return "cuda", True, gpu_name, gpu_memory
        except Exception as e:
            print(f"⚠️ GPU detected but not functional: {e}")
            print("🔄 Falling back to CPU...")
            return "cpu", False, "CPU (GPU fallback)", 0
    else:
        print("🖥️ No GPU detected - using CPU")
        return "cpu", False, "CPU", 0

# Detect device and get configuration
device, has_gpu, device_name, gpu_memory = detect_and_setup_device()

# Configure embedder with device detection
try:
    print(f"🔧 Loading SentenceTransformer on {device.upper()}...")
    embedder = SentenceTransformer(EMBED_MODEL, device=device)
    print(f"✅ SentenceTransformer loaded successfully on {device.upper()}")
except Exception as e:
    print(f"⚠️ Failed to load SentenceTransformer on {device}: {e}")
    if device == "cuda":
        print("🔄 Falling back to CPU for embedder...")
        embedder = SentenceTransformer(EMBED_MODEL, device='cpu')
        device = "cpu"
        has_gpu = False
    else:
        raise e

# Configure LLM with device detection
print(f"🤖 Loading {LLM_MODEL}")
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL, trust_remote_code=True)

try:
    if has_gpu:
        print(f"🚀 Loading model on GPU: {device_name}")
        model = AutoModelForCausalLM.from_pretrained(
            LLM_MODEL,
            trust_remote_code=True,
            device_map="cuda",
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
            use_cache=False
        )
        print("✅ Model loaded successfully on GPU")
    else:
        print("🖥️ Loading model on CPU (GPU not available)")
        model = AutoModelForCausalLM.from_pretrained(
            LLM_MODEL,
            trust_remote_code=True,
            device_map="cpu",
            torch_dtype=torch.float32,
            low_cpu_mem_usage=True,
            use_cache=False
        )
        print("✅ Model loaded successfully on CPU")
except Exception as e:
    print(f"⚠️ Failed to load model on {device}: {e}")
    if has_gpu:
        print("🔄 Falling back to CPU for model...")
        model = AutoModelForCausalLM.from_pretrained(
            LLM_MODEL,
            trust_remote_code=True,
            device_map="cpu",
            torch_dtype=torch.float32,
            low_cpu_mem_usage=True,
            use_cache=False
        )
        has_gpu = False
        device = "cpu"
        print("✅ Model loaded successfully on CPU (fallback)")
    else:
        raise e

# --- cuDF Availability Detection ---
def detect_cudf_availability():
    """Detect if cuDF is available and functional on GPU"""
    global has_gpu
    try:
        import cudf
        if has_gpu:
            # Test cuDF GPU functionality with a simple operation
            test_df = cudf.DataFrame({'test': [1, 2, 3]})
            _ = test_df.sum()
            print("✅ cuDF is available and functional on GPU")
            return True, "GPU"
        else:
            print("⚠️ cuDF available but no GPU detected - will use pandas fallback")
            return False, "CPU"
    except ImportError:
        print("⚠️ cuDF not installed - will use pandas fallback in benchmarks")
        return False, "CPU"
    except Exception as e:
        print(f"⚠️ cuDF detected but not functional on GPU: {e}")
        print("🔄 Will use pandas fallback in benchmarks")
        return False, "CPU"

# Check cuDF availability
cudf_available, cudf_device = detect_cudf_availability()

# Store device information for use throughout the application
device_info = {
    'main_device': device,
    'has_gpu': has_gpu,
    'device_name': device_name,
    'gpu_memory': gpu_memory,
    'cudf_available': cudf_available,
    'cudf_device': cudf_device
}

print(f"\n🔧 **DEVICE CONFIGURATION SUMMARY:**")
print(f"📱 Main Device: {device.upper()}")
print(f"🚀 GPU Available: {'✅ Yes' if has_gpu else '❌ No'}")
print(f"🖥️ Device Name: {device_name}")
if has_gpu:
    print(f"💾 GPU Memory: {gpu_memory:.1f} GB")
print(f"📊 cuDF Status: {'✅ Available on GPU' if cudf_available else '⚠️ Using pandas fallback'}")
print(f"🎯 Benchmarking Mode: {'Real GPU acceleration' if cudf_available else 'Simulated with pandas'}")
print("-" * 50)


llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    do_sample=True,
    return_full_text=False
) 


def stream_llm_response(prompt, max_new_tokens=1000, temperature=0.7):
    """Generate streaming response from LLM with proper token handling"""
    try:
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        
        with torch.no_grad():
            # Initialize generation parameters
            input_ids = inputs["input_ids"]
            attention_mask = inputs["attention_mask"]
            original_length = input_ids.shape[1]
            
            # Keep track of previously decoded text to extract only new content
            previous_decoded = ""
            
            # Generate tokens one by one
            for _ in range(max_new_tokens):
                outputs = model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_new_tokens=1,
                    do_sample=True,
                    temperature=temperature,
                    pad_token_id=tokenizer.eos_token_id if tokenizer.eos_token_id else tokenizer.pad_token_id,
                    return_dict_in_generate=True,
                    output_scores=True
                )
                
                # Get the new token
                new_token_id = outputs.sequences[0][-1].item()
                
                # Check for end of sequence
                if new_token_id == tokenizer.eos_token_id:
                    break
                
                # Update inputs for next generation
                input_ids = outputs.sequences
                attention_mask = torch.cat([
                    attention_mask, 
                    torch.ones((1, 1), device=model.device)
                ], dim=1)
                
                # Decode the entire new sequence and extract only the new part
                # This preserves proper spacing and formatting
                full_decoded = tokenizer.decode(
                    input_ids[0][original_length:], 
                    skip_special_tokens=True
                )
                
                # Extract only the new content since the last yield
                new_content = full_decoded[len(previous_decoded):]
                previous_decoded = full_decoded
                
                # Only yield if there's new content
                if new_content:
                    yield new_content
                
    except Exception as e:
        print(f"❌ Streaming generation failed: {e}")
        yield "Sorry, I encountered an error while generating the response."

print("✅ LLM loaded and ready!")

# === RAG UTILITIES ===

def read_urls_from_txt(path):
    """Read URLs from a text file"""
    try:
        with open(path, "r") as f:
            return [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        print(f"⚠️ URLs file not found: {path}. Creating default URLs...")
        create_default_urls_file(path)
        return read_urls_from_txt(path)


def create_default_urls_file(path):
    """Create a default URLs file with common data science resources"""
    default_urls = [
        "https://pandas.pydata.org/docs/",
        "https://docs.rapids.ai/api/cudf/stable/",
        "https://rapids.ai/",
        "https://scikit-learn.org/stable/",
        "https://pytorch.org/docs/stable/",
        "https://www.tensorflow.org/",
        "https://developer.nvidia.com/blog/tag/cuda/",
        "https://people.smp.uq.edu.au/DirkKroese/DSML/DSML.pdf"
    ]
    with open(path, "w") as f:
        for url in default_urls:
            f.write(url + "\n")
    print(f"✅ Created default URLs file: {path}")


def fetch_text_from_url(url):
    """Fetch and clean text content from a URL"""
    try:
        print(f"🌐 Fetching: {url}")
        headers = {'User-Agent': 'Mozilla/5.0 (RAG Tutor Bot)'}
        resp = requests.get(url, headers=headers, timeout=15)
        resp.raise_for_status()  # Raise exception for bad status codes
        
        # Filter out non-text content types
        content_type = resp.headers.get("Content-Type", "")
        if not content_type.startswith("text/") and "html" not in content_type:
            print(f"⚠️ Skipping non-text URL: {url} (type={content_type})")
            return ""
        
        # Parse HTML content safely
        soup = BeautifulSoup(resp.content, 'html.parser')
        for tag in soup(['script', 'style', 'header', 'footer', 'nav', 'aside']):
            tag.decompose()
        
        text = soup.get_text(separator="\n").strip()
        print(f"✅ Extracted {len(text)} characters from {url}")
        return text
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Network error fetching {url}: {e}")
        return ""
    except Exception as e:
        print(f"❌ Error processing {url}: {e}")
        return ""


def chunk_text(text, chunk_size=500, overlap=50):
    """Split text into overlapping chunks"""
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size - overlap)]


def load_all_chunks(urls):
    """Load and chunk text from all URLs"""
    all_chunks = []
    for url in urls:
        print(f"📥 Processing: {url}")
        txt = fetch_text_from_url(url)
        if txt:
            chunks = chunk_text(txt)
            all_chunks.extend(chunks)
            print(f"✅ Added {len(chunks)} chunks from {url}")
        else:
            print(f"⚠️ No content from {url}")
    return all_chunks


def embed_and_index(chunks):
    """Create embeddings and FAISS index"""
    if not chunks:
        print("⚠️ No chunks to embed")
        return None
    
    print(f"🔄 Creating embeddings for {len(chunks)} chunks...")
    vecs = embedder.encode(chunks, show_progress_bar=True, convert_to_numpy=True)
    dim = vecs.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(vecs))
    print(f"✅ Created FAISS index with {len(chunks)} chunks")
    return index


def query_rag(query, index, chunks, k=3):
    """Query the RAG system"""
    if index is None or not chunks:
        return "", ""
    
    q_vec = embedder.encode([query], convert_to_numpy=True)
    D, I = index.search(np.array(q_vec), k)
    retrieved_chunks = [chunks[i] for i in I[0] if i < len(chunks)]
    return "\n".join(retrieved_chunks), "Custom Knowledge Base"

def load_cache():
    """Load cached chunks and index"""
    if os.path.exists(CHUNK_FILE) and os.path.exists(INDEX_FILE):
        try:
            with open(CHUNK_FILE, "rb") as f:
                chunks = pickle.load(f)
            index = faiss.read_index(INDEX_FILE)
            print(f"✅ Loaded cached RAG with {len(chunks)} chunks")
            return chunks, index
        except Exception as e:
            print(f"❌ Error loading cache: {e}")
            return None, None
    return None, None

def save_cache(chunks, index):
    """Save chunks and index to cache"""
    try:
        with open(CHUNK_FILE, "wb") as f:
            pickle.dump(chunks, f)
        faiss.write_index(index, INDEX_FILE)
        print(f"✅ Cached RAG with {len(chunks)} chunks")
    except Exception as e:
        print(f"❌ Error saving cache: {e}")

def build_or_load_rag():
    """Build or load RAG system"""
    print("🔄 Initializing RAG system...")
    chunks, index = load_cache()
    
    if chunks and index is not None and len(chunks) > 0:
        print(f"✅ Using cached RAG with {len(chunks)} chunks")
        return chunks, index
    
    print("🔄 Building new RAG index...")
    
    # Check if URLs file exists
    if not os.path.exists(URLS_FILE):
        print(f"⚠️ URLs file not found: {URLS_FILE}")
        create_default_urls_file(URLS_FILE)
    
    urls = read_urls_from_txt(URLS_FILE)
    print(f"📋 Found {len(urls)} URLs to process")
    
    chunks = load_all_chunks(urls)
    print(f"📦 Total chunks extracted: {len(chunks)}")
    
    if not chunks or len(chunks) == 0:
        print("❌ No chunks loaded, RAG will be disabled")
        return [], None
    
    # Filter out very short chunks
    chunks = [chunk for chunk in chunks if len(chunk.strip()) > 50]
    print(f"📦 Filtered chunks: {len(chunks)}")
    
    if chunks:
        index = embed_and_index(chunks)
        if index is not None:
            save_cache(chunks, index)
            print(f"✅ RAG system built successfully with {len(chunks)} chunks")
            return chunks, index
        else:
            print("❌ Failed to create embeddings")
            return [], None
    else:
        print("⚠️ No valid chunks after filtering, RAG will be disabled")
        return [], None

def is_garbage(text):
    """Check if text is low quality or corrupted"""
    if not text or len(text) < 80:
        return True
    non_ascii = sum(1 for c in text if ord(c) > 126 or ord(c) < 9)
    return non_ascii / len(text) > 0.2


# Initialize RAG system
print("🚀 Initializing RAG system...")
rag_chunks, rag_index = build_or_load_rag()

# === CORE FUNCTIONS ===

def generate_contextual_quote(context="learning", user_query=""):
    """Generate contextual inspirational quotes from quotes.txt file"""
    
    def format_quote_html(quote_text, author="Data Science Wisdom"):
        """Format a quote into HTML with proper styling"""
        content = quote_text.strip()
        
        return f"""
        <div class="inspirational-quote">
            <div class="quote-content">{content}</div>
            <div class="quote-author">— {author}</div>
        </div>
        """
    
    def load_quotes_from_file():
        """Load quotes from the quotes.txt file"""
        quotes_file = "quotes.txt"
        try:
            with open(quotes_file, "r", encoding="utf-8") as f:
                quotes = [line.strip() for line in f if line.strip()]
                return quotes
        except FileNotFoundError:
            print(f"⚠️ Quotes file not found: {quotes_file}")
            return None
    
    # Load quotes from file
    file_quotes = load_quotes_from_file()
    
    if file_quotes:
        # Randomly select a quote from the file
        selected_quote = random.choice(file_quotes)
        return format_quote_html(selected_quote)
    
    # Fallback quotes if file not found
    fallback_quotes = [
        "Data science is not just about the algorithms, it's about extracting insights that drive decisions.",
        "GPU acceleration transforms impossible computations into everyday possibilities.",
        "The best data scientists are those who can bridge the gap between data and actionable insights.",
        "Machine learning is statistics on steroids, powered by GPU acceleration.",
        "Data without context is just noise; context without data is just opinion."
    ]
    
    selected_quote = random.choice(fallback_quotes)
    return format_quote_html(selected_quote)


def extract_topic_from_query(query):
    """Extract main topic from user query"""
    if not query:
        return "learning"
    
    query_lower = query.lower()
    topic_keywords = {
        'gpu computing': ['gpu', 'cuda', 'parallel', 'graphics'],
        'data science': ['data', 'science', 'analytics', 'statistics'],
        'machine learning': ['machine learning', 'ml', 'ai', 'artificial intelligence'],
        'pandas': ['pandas', 'dataframe'],
        'rapids': ['rapids', 'cudf', 'cuml'],
        'programming': ['python', 'code', 'programming']
    }
    
    for topic, keywords in topic_keywords.items():
        if any(keyword in query_lower for keyword in keywords):
            return topic
    return 'learning'


def smart_tutor_answer_streaming(query):
    """Enhanced RAG-powered Socratic tutor responses with streaming"""
    if not query or query.strip() == "":
        yield "🤔 Please ask me a question about data science, machine learning, or GPU computing!"
        return
    
    # Try RAG first
    if rag_index is not None and rag_chunks:
        context, source = query_rag(query, rag_index, rag_chunks)
        
        if context and not is_garbage(context):
            # Stream a short semi-Socratic response
            polished_prompt = f"""You are a semi-Socratic tutor. Use this context to guide the learner:

Context: {context[:300]}

Rules:
- Ask 1-2 guiding questions first
- Give a short, clear answer (2-3 sentences max)
- Ask one follow-up question to deepen understanding
- Keep total response under 100 words

Topic: {query}
Response:"""
            for token in stream_llm_response(polished_prompt, max_new_tokens=150, temperature=0.6):
                yield token
            return
    
    # Fallback to LLM streaming for non-RAG responses
    query_lower = query.lower()

    # Generic response with LLM streaming
    if llm_pipeline:
        try:
            prompt = f"""You are a semi-Socratic tutor. Be concise and guide discovery.

Rules:
- Ask 1 thought-provoking question first
- Give a brief, clear answer (2-3 sentences)
- End with one follow-up question
- Total response: under 80 words

Student asks: {query}
Your response:"""
            for token in stream_llm_response(prompt, max_new_tokens=200, temperature=0.6):
                yield token
            return
        except Exception as e:
            print(f"❌ LLM fallback failed: {e}")

    yield f"""🤔 **Let's think about this together:**

• What do you already know about {query}?
• How might this help solve real problems?

💡 **Quick hint:** Think about the core concept first, then we can dive deeper!"""

def generate_flashcards(topic, n=3):
    """Generate educational flashcards with RAG enhancement"""
    
    # Try RAG-based generation first
    if rag_index is not None and rag_chunks:
        try:
            context, source = query_rag(topic, rag_index, rag_chunks)
            if context and len(context.strip()) > 100:
                prompt = f"""
Create {n} educational flashcards about {topic} from this content. Format as JSON array:
[{{"front": "question", "back": "answer"}}]

Content:
{context[:1000]}
"""
                result = llm_pipeline(prompt, max_new_tokens=1000, temperature=0.7)
                if result:
                    text = result[0]['generated_text']
                    # Try to extract JSON array
                    match = re.search(r'\[.*\]', text, re.DOTALL)
                    if match:
                        try:
                            cards = ast.literal_eval(match.group(0))
                            if isinstance(cards, list) and len(cards) > 0:
                                return cards[:n]
                        except:
                            pass
        except Exception as e:
            print(f"❌ RAG flashcard generation failed: {e}")
    
    # Fallback flashcards
    flashcard_pools = {
        "cudf": [
            {"front": "What is cuDF?", "back": "cuDF is a GPU DataFrame library with pandas-like API."},
            {"front": "How to convert pandas to cuDF?", "back": "Use cudf.from_pandas(df)"},
            {"front": "Main advantage of cuDF?", "back": "GPU acceleration for large dataset processing"}
        ],
        "pandas": [
            {"front": "What is a DataFrame?", "back": "A 2D labeled data structure with columns of different types."},
            {"front": "How to read CSV in pandas?", "back": "pd.read_csv('filename.csv')"},
            {"front": "How to select a column?", "back": "df['column_name'] or df.column_name"}
        ],
        "rapids": [
            {"front": "What is RAPIDS?", "back": "A suite of GPU-accelerated data science libraries."},
            {"front": "Main RAPIDS libraries?", "back": "cuDF, cuML, cuGraph for DataFrames, ML, and graphs."},
            {"front": "RAPIDS vs traditional tools?", "back": "Same APIs but with GPU acceleration for massive speedups."}
        ],
        "cuda": [
            {"front": "What does CUDA stand for?", "back": "Compute Unified Device Architecture"},
            {"front": "What is CUDA used for?", "back": "Parallel computing on NVIDIA GPUs"},
            {"front": "CUDA vs CPU computing?", "back": "CUDA enables massive parallelization vs sequential CPU processing"}
        ]
    }
    
    # Get cards for topic, with fallback
    cards = flashcard_pools.get(topic.lower(), [
        {"front": f"What is {topic}?", "back": f"{topic} is an important concept in data science."},
        {"front": f"Why learn {topic}?", "back": f"Understanding {topic} improves your data science skills."}
    ])
    
    return cards[:n]

def generate_quiz(topic, n=2):
    """Generate quiz questions"""
    
    # Quiz pools by topic
    quiz_pools = {
        "pandas": [
            {"question": "What is the main data structure in pandas?", "options": ["DataFrame", "Array", "List", "Dict"], "answer_idx": 0},
            {"question": "How do you read a CSV file?", "options": ["pd.read_csv()", "pd.load()", "pd.import()", "pd.open()"], "answer_idx": 0}
        ],
        "cudf": [
            {"question": "cuDF accelerates which library?", "options": ["NumPy", "Pandas", "SciPy", "Matplotlib"], "answer_idx": 1},
            {"question": "cuDF runs on which hardware?", "options": ["CPU", "GPU", "TPU", "FPGA"], "answer_idx": 1}
        ],
        "rapids": [
            {"question": "RAPIDS is developed by?", "options": ["Google", "Facebook", "NVIDIA", "Microsoft"], "answer_idx": 2},
            {"question": "Main RAPIDS component for DataFrames?", "options": ["cuML", "cuDF", "cuGraph", "cuPy"], "answer_idx": 1}
        ]
    }
    
    questions = quiz_pools.get(topic.lower(), [
        {"question": f"What is {topic}?", "options": ["A tool", "A library", "A concept", "All of above"], "answer_idx": 3},
        {"question": f"Why is {topic} important?", "options": ["Performance", "Efficiency", "Scale", "All of above"], "answer_idx": 3}
    ])
    
    return random.sample(questions, min(n, len(questions)))


def generate_coding_puzzle(difficulty="Beginner"):
    """Generate coding puzzles for pandas to cuDF conversion"""
    
    puzzles = {
        "Beginner": {
            "description": "Convert basic DataFrame creation from pandas to cuDF",
            "cpu_code": "import pandas as pd\ndf = pd.DataFrame({'a': [1, 2, 3]})\nprint(df)",
            "gpu_code": "import cudf\ndf = cudf.DataFrame({'a': [1, 2, 3]})\nprint(df)",
            "cpu_time": 1.2,
            "gpu_time": 0.2
        },
        "Intermediate": {
            "description": "Convert groupby operation from pandas to cuDF",
            "cpu_code": "import pandas as pd\ndf = pd.DataFrame({'group': ['A', 'B', 'A'], 'value': [1, 2, 3]})\nresult = df.groupby('group').sum()",
            "gpu_code": "import cudf\ndf = cudf.DataFrame({'group': ['A', 'B', 'A'], 'value': [1, 2, 3]})\nresult = df.groupby('group').sum()",
            "cpu_time": 3.2,
            "gpu_time": 0.5
        },
        "Advanced": {
            "description": "Convert complex aggregation from pandas to cuDF",
            "cpu_code": "import pandas as pd\ndf = pd.DataFrame({'cat': ['A', 'B'], 'val1': [1, 2], 'val2': [3, 4]})\nresult = df.groupby('cat').agg({'val1': 'sum', 'val2': 'mean'})",
            "gpu_code": "import cudf\ndf = cudf.DataFrame({'cat': ['A', 'B'], 'val1': [1, 2], 'val2': [3, 4]})\nresult = df.groupby('cat').agg({'val1': 'sum', 'val2': 'mean'})",
            "cpu_time": 4.2,
            "gpu_time": 0.6
        }
    }
    
    return puzzles.get(difficulty, puzzles["Beginner"])


def create_speedup_visualization(cpu_time, gpu_time, speedup):
    """Create a visual ASCII-style chart for speedup comparison"""
    max_width = 50
    cpu_bar = "█" * max_width
    gpu_bar = "█" * max(1, int(max_width / speedup))
    
    chart = f"""
```
PERFORMANCE COMPARISON CHART
════════════════════════════════════════════════════════════

🖥️  CPU (pandas):  {cpu_bar} {cpu_time}s
🚀  GPU (cuDF):    {gpu_bar} {gpu_time}s

SPEEDUP: {speedup}x faster! 🔥
════════════════════════════════════════════════════════════
```
"""
    return chart


# === GRADIO INTERFACE ===
def create_ai_tutor():
    """Create the AI Tutor Gradio interface"""
    
    with gr.Blocks(theme='earneleh/paris', title="🧠 AI Tutor") as app:
        
        # Header
        gr.HTML("""
        <div style='text-align:center; background:linear-gradient(90deg,#fff1c1,#c1e7ff,#e1ffc1); border-radius:15px; padding:15px; margin-bottom:20px; color:#000;'>
          <h1 style='color:#000; text-shadow: 1px 1px 2px rgba(0,0,0,0.1);'>🧠 AI Tutor - Complete Learning System</h1>
          <h3 style='color:#000; font-weight:700; text-shadow: 1px 1px 2px rgba(0,0,0,0.1);'>📚 Learning Mode • 🎓 Socratic Tutoring • 🎮 Gamified Practice</h3>
          <p>
            <b style='color:#000; font-weight:600; text-shadow: 1px 1px 2px rgba(0,0,0,0.1);'>📚 Structured Learning Paths • 🤔 Guided Discovery • 🎯 Interactive Practice</b>
          </p>
        </div>
        """)
        
        # Add custom CSS for better styling
        app.css = """
        .learning-topic-btn {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
            color: white !important;
            border: none !important;
            border-radius: 12px !important;
            padding: 15px !important;
            margin: 8px !important;
            font-weight: 600 !important;
            box-shadow: 0 4px 15px rgba(102, 126, 234, 0.4) !important;
            transition: all 0.3s ease !important;
            min-height: 80px !important;
            font-size: 14px !important;
        }
        .learning-topic-btn:hover {
            transform: translateY(-2px) !important;
            box-shadow: 0 6px 20px rgba(102, 126, 234, 0.6) !important;
        }
        
        /* Make tab headers more prominent */
        .gradio-tab-nav {
            background: linear-gradient(90deg, #f8fafc, #e2e8f0) !important;
            border-radius: 8px !important;
            padding: 8px !important;
            margin-bottom: 20px !important;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1) !important;
        }
        
        .gradio-tab-nav .tab-nav {
            background: linear-gradient(135deg, #4f46e5, #7c3aed) !important;
            color: white !important;
            font-weight: 700 !important;
            font-size: 16px !important;
            border-radius: 6px !important;
            margin: 4px !important;
            padding: 12px 20px !important;
            box-shadow: 0 2px 4px rgba(79, 70, 229, 0.3) !important;
            border: none !important;
        }
        
        .gradio-tab-nav .tab-nav:hover {
            background: linear-gradient(135deg, #6366f1, #8b5cf6) !important;
            transform: translateY(-1px) !important;
            box-shadow: 0 4px 8px rgba(79, 70, 229, 0.4) !important;
        }
        
        .gradio-tab-nav .tab-nav.selected {
            background: linear-gradient(135deg, #059669, #0d9488) !important;
            box-shadow: 0 4px 12px rgba(5, 150, 105, 0.4) !important;
        }
        
        /* Benchmark button styling */
        .benchmark-btn {
            background: linear-gradient(135deg, #ff6b35, #f7931e) !important;
            color: white !important;
            font-weight: 700 !important;
            font-size: 18px !important;
            padding: 15px 30px !important;
            border-radius: 12px !important;
            border: none !important;
            box-shadow: 0 4px 15px rgba(255, 107, 53, 0.4) !important;
            transition: all 0.3s ease !important;
            text-transform: uppercase !important;
            letter-spacing: 1px !important;
        }
        
        .benchmark-btn:hover {
            background: linear-gradient(135deg, #ff8c5a, #ffb347) !important;
            transform: translateY(-2px) !important;
            box-shadow: 0 6px 20px rgba(255, 107, 53, 0.6) !important;
        }
        .textBoxColor textarea {
            background-color: transparent !important;
            color: ;
        }
        
        /* Quote display styling */
        .inspirational-quote {
            background: linear-gradient(135deg, #fef7cd, #fbbf24, #f59e0b) !important;
            border: none !important;
            border-radius: 15px !important;
            padding: 20px !important;
            margin: 10px 0 !important;
            box-shadow: 0 8px 25px rgba(245, 158, 11, 0.3) !important;
            text-align: center !important;
            font-family: 'Georgia', 'Times New Roman', serif !important;
            font-style: italic !important;
            font-size: 18px !important;
            line-height: 1.6 !important;
            color: #92400e !important;
            position: relative !important;
            overflow: hidden !important;
        }
        
        .inspirational-quote::before {
            content: '"' !important;
            font-size: 60px !important;
            color: rgba(146, 64, 14, 0.2) !important;
            position: absolute !important;
            top: -10px !important;
            left: 15px !important;
            font-family: 'Georgia', serif !important;
        }
        
        .inspirational-quote::after {
            content: '"' !important;
            font-size: 60px !important;
            color: rgba(146, 64, 14, 0.2) !important;
            position: absolute !important;
            bottom: -35px !important;
            right: 15px !important;
            font-family: 'Georgia', serif !important;
        }
        
        .quote-content {
            position: relative !important;
            z-index: 1 !important;
            font-weight: 500 !important;
            text-shadow: 1px 1px 2px rgba(0,0,0,0.1) !important;
        }
        
        .quote-author {
            margin-top: 15px !important;
            font-size: 14px !important;
            font-weight: 600 !important;
            color: #78350f !important;
            font-style: normal !important;
        }
        """
        
        with gr.Tabs():
            
            # === LEARNING MODE ===
            with gr.Tab("📚 Learning Mode"):
                gr.Markdown("### 🎯 **Choose a Learning Path - From Beginner to Advanced**")
                gr.Markdown("*Select any topic below to get a structured learning journey with resources and related topics*")
                
                # Learning topics with structured paths
                LEARNING_TOPICS = {
                    "Python for Data Science": {
                        "description": "Master Python fundamentals and data science libraries",
                        "level": "🟢 Beginner to Intermediate",
                        "duration": "4-6 weeks",
                        "prerequisites": "Basic programming knowledge",
                        "content": """
**📖 Learning Path:**
1. **Python Basics** - Variables, data types, control structures
2. **NumPy** - Numerical computing and arrays
3. **Pandas** - Data manipulation and analysis
4. **Matplotlib/Seaborn** - Data visualization
5. **Jupyter Notebooks** - Interactive development

**🔗 Key Resources:**
• [Python.org Tutorial](https://docs.python.org/3/tutorial/)
• [Pandas Documentation](https://pandas.pydata.org/docs/)
• [NumPy User Guide](https://numpy.org/doc/stable/user/)

**🚀 Next Steps:** Machine Learning Fundamentals, Data Visualization
""",
                        "related": ["Machine Learning Fundamentals", "Data Visualization", "Pandas Deep Dive"]
                    },
                    "Pandas Deep Dive": {
                        "description": "Master DataFrame operations, data cleaning, and advanced pandas techniques",
                        "level": "🟡 Intermediate",
                        "duration": "3-4 weeks", 
                        "prerequisites": "Python basics, basic pandas knowledge",
                        "content": """
**📖 Learning Path:**
1. **DataFrame Mastery** - Creation, indexing, selection
2. **Data Cleaning** - Missing values, duplicates, data types
3. **GroupBy Operations** - Aggregation and transformation
4. **Merging & Joining** - Combining datasets
5. **Performance Optimization** - Efficient pandas operations

**🔗 Key Resources:**
• [Pandas Documentation](https://pandas.pydata.org/docs/)
• [Pandas Cookbook](https://pandas.pydata.org/docs/user_guide/cookbook.html)
• [10 Minutes to Pandas](https://pandas.pydata.org/docs/user_guide/10min.html)

**🚀 Next Steps:** GPU Acceleration with cuDF, Time Series Analysis
""",
                        "related": ["GPU Acceleration with cuDF", "Data Visualization", "Time Series Analysis"]
                    },
                    "GPU Acceleration with cuDF": {
                        "description": "Learn GPU-accelerated data processing with RAPIDS cuDF",
                        "level": "🔴 Advanced",
                        "duration": "2-3 weeks",
                        "prerequisites": "Strong pandas knowledge, basic GPU concepts",
                        "content": """
**📖 Learning Path:**
1. **GPU Computing Basics** - Understanding parallel processing
2. **cuDF Introduction** - GPU DataFrames and basic operations
3. **Migration from Pandas** - Converting existing code
4. **Performance Optimization** - Memory management and best practices
5. **Advanced Operations** - Complex aggregations and joins

**🔗 Key Resources:**
• [RAPIDS cuDF Documentation](https://docs.rapids.ai/api/cudf/stable/)
• [cuDF User Guide](https://docs.rapids.ai/api/cudf/stable/user_guide/)
• [RAPIDS Getting Started](https://rapids.ai/start.html)

**🚀 Next Steps:** RAPIDS Ecosystem, Machine Learning with cuML
""",
                        "related": ["RAPIDS Ecosystem", "CUDA Programming", "High-Performance Computing"]
                    },
                    "Machine Learning Fundamentals": {
                        "description": "Core ML concepts, algorithms, and scikit-learn implementation",
                        "level": "🟡 Intermediate",
                        "duration": "6-8 weeks",
                        "prerequisites": "Python, pandas, basic statistics",
                        "content": """
**📖 Learning Path:**
1. **ML Concepts** - Supervised, unsupervised, reinforcement learning
2. **Data Preprocessing** - Feature scaling, encoding, train-test splits
3. **Regression Algorithms** - Linear, polynomial, regularization
4. **Classification** - Logistic regression, decision trees, SVM
5. **Model Evaluation** - Cross-validation, metrics, hyperparameter tuning

**🔗 Key Resources:**
• [Scikit-learn Documentation](https://scikit-learn.org/stable/)
• [Machine Learning Course by Andrew Ng](https://www.coursera.org/learn/machine-learning)
• [Hands-On Machine Learning](https://github.com/ageron/handson-ml2)

**🚀 Next Steps:** Deep Learning, GPU-Accelerated ML with cuML
""",
                        "related": ["Deep Learning Fundamentals", "GPU-Accelerated ML", "Data Science Projects"]
                    },
                    "RAPIDS Ecosystem": {
                        "description": "Complete RAPIDS suite: cuDF, cuML, cuGraph for end-to-end GPU data science",
                        "level": "🔴 Advanced",
                        "duration": "4-5 weeks",
                        "prerequisites": "GPU computing basics, pandas, scikit-learn",
                        "content": """
**📖 Learning Path:**
1. **RAPIDS Overview** - cuDF, cuML, cuGraph, cuPy integration
2. **cuDF Mastery** - Advanced DataFrame operations on GPU
3. **cuML for ML** - GPU-accelerated machine learning algorithms
4. **cuGraph** - Graph analytics and network analysis
5. **End-to-End Workflows** - Complete GPU data science pipelines

**🔗 Key Resources:**
• [RAPIDS.ai Main Site](https://rapids.ai/)
• [RAPIDS Documentation](https://docs.rapids.ai/)
• [RAPIDS Community](https://github.com/rapidsai)

**🚀 Next Steps:** Production Deployment, Multi-GPU Computing
""",
                        "related": ["Multi-GPU Computing", "Production ML Systems", "CUDA Programming"]
                    },
                    "Data Visualization": {
                        "description": "Create compelling data visualizations with matplotlib, seaborn, and plotly",
                        "level": "🟢 Beginner to Intermediate",
                        "duration": "3-4 weeks",
                        "prerequisites": "Python basics, pandas fundamentals",
                        "content": """
**📖 Learning Path:**
1. **Matplotlib Basics** - Plots, figures, axes, customization
2. **Seaborn for Statistics** - Statistical plots and themes
3. **Interactive Plotly** - Dynamic and web-ready visualizations
4. **Advanced Techniques** - Subplots, animations, custom plots
5. **Dashboard Creation** - Streamlit, Dash for interactive apps

**🔗 Key Resources:**
• [Matplotlib Documentation](https://matplotlib.org/stable/contents.html)
• [Seaborn Tutorial](https://seaborn.pydata.org/tutorial.html)
• [Plotly Python Guide](https://plotly.com/python/)

**🚀 Next Steps:** Interactive Dashboards, Business Intelligence
""",
                        "related": ["Interactive Dashboards", "Business Intelligence", "Web Development for Data Science"]
                    }
                }
                
                # Topic selection buttons
                with gr.Row():
                    topic_buttons = []
                    for i, (topic_name, topic_info) in enumerate(LEARNING_TOPICS.items()):
                        if i % 2 == 0 and i > 0:
                            # Start new row every 2 buttons
                            pass
                        
                        color_map = {
                            "🟢": "#dcfce7",  # Green for beginner
                            "🟡": "#fef3c7",  # Yellow for intermediate  
                            "🔴": "#fee2e2"   # Red for advanced
                        }
                        level_color = color_map.get(topic_info["level"][0], "#f3f4f6")
                        
                        btn = gr.Button(
                            f"{topic_name}\n{topic_info['level']}",
                            elem_classes="learning-topic-btn",
                            size="lg"
                        )
                        topic_buttons.append((btn, topic_name))
                
                # Learning content display
                learning_content = gr.Markdown()
                related_topics = gr.Markdown()
                
                def show_learning_content(topic_name):
                    if topic_name not in LEARNING_TOPICS:
                        return "Topic not found!", ""
                    
                    topic = LEARNING_TOPICS[topic_name]
                    
                    content = f"""
# 📚 {topic_name}

**📋 Description:** {topic['description']}

**📊 Level:** {topic['level']} | **⏱️ Duration:** {topic['duration']} | **📚 Prerequisites:** {topic['prerequisites']}

---

{topic['content']}

---
                    """
                    
                    related = f"""
### 🔗 **Related Learning Paths:**
{' • '.join([f"**{rel}**" for rel in topic['related']])}

💡 *Click any topic above to explore these related learning paths!*
                    """
                    
                    return content, related
                
                # Connect buttons to content display
                for btn, topic_name in topic_buttons:
                    btn.click(
                        lambda tn=topic_name: show_learning_content(tn),
                        outputs=[learning_content, related_topics]
                    )
                
                # Search functionality
                with gr.Row():
                    search_topic = gr.Textbox(
                        label="🔍 Search for specific topics",
                        placeholder="e.g., neural networks, time series, NLP, computer vision...",
                        elem_classes="textBoxColor"
                    )
                    search_btn = gr.Button("Search Learning Resources")
                
                search_results = gr.Markdown()
                
                def search_learning_resources(query):
                    if not query:
                        return "Please enter a search term!"
                    
                    # Simulate search results with relevant resources
                    query_lower = query.lower()
                    
                    results = []
                    
                    if any(term in query_lower for term in ['neural', 'deep', 'cnn', 'rnn', 'transformer']):
                        results.append("""
**🧠 Deep Learning Resources:**
• [Deep Learning Specialization](https://www.coursera.org/specializations/deep-learning)
• [PyTorch Tutorials](https://pytorch.org/tutorials/)
• [TensorFlow Guide](https://www.tensorflow.org/guide)
""")
                    
                    if any(term in query_lower for term in ['nlp', 'text', 'language', 'sentiment']):
                        results.append("""
**📝 Natural Language Processing:**
• [NLTK Documentation](https://www.nltk.org/)
• [spaCy Course](https://course.spacy.io/)
• [Hugging Face Transformers](https://huggingface.co/transformers/)
""")
                    
                    if any(term in query_lower for term in ['time series', 'forecasting', 'temporal']):
                        results.append("""
**📈 Time Series Analysis:**
• [Time Series Analysis Guide](https://www.statsmodels.org/stable/tsa.html)
• [Prophet Forecasting](https://facebook.github.io/prophet/)
• [Time Series with Python](https://github.com/marcopeix/TimeSeriesForecastingInPython)
""")
                    
                    if any(term in query_lower for term in ['computer vision', 'image', 'cv', 'opencv']):
                        results.append("""
**👁️ Computer Vision:**
• [OpenCV Tutorials](https://docs.opencv.org/master/d9/df8/tutorial_root.html)
• [Computer Vision Course](https://www.coursera.org/learn/convolutional-neural-networks)
• [Fast.ai Practical Deep Learning](https://course.fast.ai/)
""")
                    
                    if not results:
                        results.append(f"""
**🔍 Search Results for "{query}":**

*No specific resources found. Try these general resources:*
• [Kaggle Learn](https://www.kaggle.com/learn) - Free micro-courses
• [Coursera Data Science](https://www.coursera.org/browse/data-science)
• [edX MIT Data Science](https://www.edx.org/course/introduction-to-computational-thinking-and-data-science)

*Or refine your search with terms like: machine learning, deep learning, NLP, computer vision, time series*
""")
                    
                    return "\n".join(results)
                
                search_btn.click(search_learning_resources, inputs=search_topic, outputs=search_results)

            # === TUTOR MODE ===
            with gr.Tab("🎓 Tutor Mode"):
                gr.Markdown("### 🤔 **Socratic Learning - Ask Questions, Get Guided Answers**")
                gr.Markdown("*I won't give you direct answers, but I'll guide you to discover the knowledge yourself!*")
                
                with gr.Row():
                    with gr.Column(scale=2):
                        tutor_query = gr.Textbox(
                            label="Ask about Data Science, GPU Computing, or type 'docs' for documentation",
                            placeholder="e.g., What is RAPIDS? How does cuDF work? Why use GPU for data science?",
                            lines=2,
                            elem_classes="textBoxColor"
                        )
                        tutor_btn = gr.Button("🚀 Get Socratic Guidance", variant="primary")
                    
                    with gr.Column(scale=1):
                        quote_display = gr.HTML("""
                        <div class="inspirational-quote">
                            <div class="quote-content">💡 Ready to discover knowledge!</div>
                            <div class="quote-author">— AI Tutor</div>
                        </div>
                        """, elem_classes="inspirational-quote")
                
                tutor_answer = gr.Markdown(label="🤔 Socratic Guidance")
                tutor_source = gr.Markdown(label="📚 Source")
                
                # Quick question suggestions
                with gr.Row():
                    quick_questions = [
                        "Why use GPUs for data science?",
                        "How does parallel processing work?", 
                        "What makes cuDF faster than pandas?",
                        "When should I use machine learning?"
                    ]
                    for i, question in enumerate(quick_questions):
                        if i % 2 == 0:
                            with gr.Column():
                                pass
                        quick_btn = gr.Button(f"💭 {question}", size="sm")
                        quick_btn.click(lambda q=question: q, outputs=tutor_query)
                
                def handle_tutor_query_streaming(query):
                    if not query:
                        ready_quote = """
                        <div class="inspirational-quote">
                            <div class="quote-content">Ready to help you discover knowledge!</div>
                            <div class="quote-author">— AI Tutor</div>
                        </div>
                        """
                        yield "Please ask a question!", "AI Tutor", ready_quote
                        return
                    
                    # Handle docs command
                    if query.lower().strip() in ['docs', 'documentation', 'help']:
                        docs_info = """📚 **Available Documentation:**
• **Core Libraries:** pandas, cuDF, RAPIDS, scikit-learn
• **GPU Computing:** CUDA, NVIDIA resources  
• **Data Science:** Comprehensive guides and tutorials
*Ask any topic-specific question to get relevant documentation!*"""
                        docs_quote = """
                        <div class="inspirational-quote">
                            <div class="quote-content">Knowledge at your fingertips!</div>
                            <div class="quote-author">— Documentation System</div>
                        </div>
                        """
                        yield docs_info, "Documentation System", docs_quote
                        return
                    
                    # Generate contextual quote
                    quote = generate_contextual_quote(extract_topic_from_query(query), query)
                    
                    # Stream the response
                    accumulated_response = ""
                    for token in smart_tutor_answer_streaming(query):
                        accumulated_response += token
                        yield accumulated_response, f"**Source:** AI Tutor (Streaming)", quote
                
                # Update the click handlers to use streaming
                tutor_btn.click(
                    handle_tutor_query_streaming, 
                    inputs=tutor_query, 
                    outputs=[tutor_answer, tutor_source, quote_display],
                    show_progress=True
                )
                tutor_query.submit(
                    handle_tutor_query_streaming, 
                    inputs=tutor_query, 
                    outputs=[tutor_answer, tutor_source, quote_display],
                    show_progress=True
                )
            
            # === GAME MODE ===
            with gr.Tab("🎮 Game Mode"):
                user_state = SimpleNamespace(points=0)
                
                def update_progress():
                    level = user_state.points // 20 + 1
                    bar_width = min((user_state.points % 20) * 5, 100)
                    return f"""<h3>🏆 Level {level} | 🔥 {user_state.points} XP</h3>
                    <div style='background: #e5e7eb; height: 20px; border-radius: 10px;'>
                        <div style='background: linear-gradient(90deg, #4ade80, #22c55e); height: 20px; width: {bar_width}%; border-radius: 10px;'></div>
                    </div>"""
                
                progress = gr.HTML(update_progress())
                
                #with gr.Tabs():
                    
                # === FLASHCARDS ===
                with gr.Tabs():
                    with gr.Tab("🃏 Flashcards"):
                        with gr.Tab("🔧 Generate Flashcards"):
                            gen_query = gr.Textbox(label="Enter a topic (e.g., cuDF, CUDA, RAPIDS)", placeholder="cuDF vs pandas?",  elem_classes="textBoxColor")
                            gen_btn = gr.Button("✨ Generate Flashcards")
                
                            card_front = gr.HTML()
                            card_back = gr.HTML(visible=False)
                            instruction_msg = gr.Markdown("👆 Choose a card to flip!", visible=False)
                            feedback_msg = gr.Markdown(visible=False)
                
                            with gr.Row():
                                flip_btn1 = gr.Button("✅ Flip Card 1", visible=False)
                                flip_btn2 = gr.Button("🔄 Flip Card 2", visible=False)
                
                            reset_btn = gr.Button("🔁 Reset", visible=False)
                
                            flash_idx = gr.State(0)
                            show_answer = gr.State(False)
                            flashcards_state = gr.State([])
                
                            import ast
                            import random
                            import re
                            def generate_flashcards_from_rag(query, index, chunks, k=3):
                                context, source = query_rag(query, index, chunks)
                
                                if not context or len(context.strip()) < 100:
                                    return [{"front": "Could not find content", "back": "Try another topic.", "wrong": "This is a placeholder wrong answer."}]
                
                                prompt = f"""
You're an expert AI tutor. From the following technical text, generate exactly {k} multiple-choice style flashcards in **valid JSON**.

Each flashcard must be a dictionary with:
- "front": the question
- "back": the correct answer
- "wrong": a wrong but plausible distractor

Example format:

[
  {{
    "front": "Why is cuDF faster than pandas?",
    "back": "It uses GPU acceleration.",
    "wrong": "Because it uses more RAM."
  }},
  {{
    "front": "What does GPU stand for?",
    "back": "Graphics Processing Unit.",
    "wrong": "General Purpose Unit."
  }}
]

CRITICAL: Output ONLY the JSON array. No explanations, no markdown, no extra text.

Text:
------------------------
{context[:800]}
------------------------
"""
                            
                                try:
                                    raw = llm_pipeline(prompt)[0]['generated_text']
                            
                                    # DEBUG: Log raw output when things fail
                                    print(f"\n🔍 RAW OUTPUT FROM MODEL ({len(raw)} chars):\n{raw}\n")
                            
                                    # Multiple parsing strategies
                                    parsed_cards = None
                                    
                                    # Strategy 1: Direct JSON parsing
                                    try:
                                        import json
                                        # Try to find JSON in the raw output
                                        lines = raw.strip().split('\n')
                                        for i, line in enumerate(lines):
                                            if line.strip().startswith('['):
                                                json_candidate = '\n'.join(lines[i:])
                                                # Clean up common issues
                                                json_candidate = json_candidate.split('```')[0]  # Remove markdown
                                                json_candidate = re.sub(r'}\s*,\s*]', '}]', json_candidate)  # Fix trailing commas
                                                parsed_cards = json.loads(json_candidate)
                                                break
                                    except Exception as e:
                                        print(f"Strategy 1 failed: {e}")
                                    
                                    # Strategy 2: Regex extraction with improved pattern
                                    if not parsed_cards:
                                        try:
                                            # More flexible regex that handles whitespace and newlines
                                            json_pattern = r'\[\s*(?:\{[^}]*\}\s*,?\s*)+\]'
                                            json_matches = re.findall(json_pattern, raw, re.DOTALL)
                                            if json_matches:
                                                json_text = json_matches[0]
                                                # Clean the JSON
                                                json_text = re.sub(r'}\s*,\s*]', '}]', json_text)  # Fix trailing commas
                                                json_text = re.sub(r',\s*}', '}', json_text)  # Fix trailing commas in objects
                                                parsed_cards = json.loads(json_text)
                                        except Exception as e:
                                            print(f"Strategy 2 failed: {e}")
                                    
                                    # Strategy 3: ast.literal_eval fallback
                                    if not parsed_cards:
                                        try:
                                            json_text = re.search(r'\[\s*\{.*?\}\s*\]', raw, re.DOTALL)
                                            if json_text:
                                                parsed_cards = ast.literal_eval(json_text.group())
                                        except Exception as e:
                                            print(f"Strategy 3 failed: {e}")
                                    
                                    # Validate the parsed cards
                                    if parsed_cards and isinstance(parsed_cards, list) and len(parsed_cards) > 0:
                                        valid_cards = []
                                        for card in parsed_cards:
                                            if isinstance(card, dict) and 'front' in card and 'back' in card:
                                                # Ensure all required fields exist
                                                valid_card = {
                                                    'front': str(card.get('front', 'Generated question')),
                                                    'back': str(card.get('back', 'Generated answer')),
                                                    'wrong': str(card.get('wrong', 'Wrong answer'))
                                                }
                                                valid_cards.append(valid_card)
                                        
                                        if valid_cards:
                                            print(f"✅ Successfully parsed {len(valid_cards)} flashcards")
                                            return valid_cards[:k]  # Return only requested number
                                    
                                    print("❌ All parsing strategies failed")
                                    
                                except Exception as e:
                                    print(f"❌ Flashcard generation failed: {e}")
                                    import traceback
                                    traceback.print_exc()
                
                                # Enhanced fallback cards with the topic
                                print(f"🔄 Using fallback flashcards for topic: {query}")
                                fallback_cards = [
                                    {"front": f"What is {query}?", "back": f"{query} is an important technology in data science.", "wrong": f"{query} is outdated technology."},
                                    {"front": f"Why is {query} useful?", "back": f"{query} helps improve performance and efficiency.", "wrong": f"{query} is only useful for beginners."},
                                    {"front": f"How does {query} work?", "back": f"{query} uses advanced algorithms and optimization.", "wrong": f"{query} works by slowing down processes."}
                                ]
                                return fallback_cards[:k]
                
                            def style_card(text):
                                return f"""
                <div style='font-size:18px;padding:10px;border:2px solid #ccc;border-radius:10px;background:#fefefe;transition:transform 0.5s;transform-style: preserve-3d;'>
                {text}
                </div>"""
                
                            def flashcard_from_query(topic, idx):
                                if not topic or topic.strip() == "":
                                    return (
                                            style_card("**Q:** Please enter a topic to generate flashcards."),
                                            "",
                                            gr.update(visible=False),
                                            gr.update(visible=False),
                                            gr.update(visible=False),
                                            gr.update(visible=False),
                                            gr.update(visible=True, value="<span style='color:red; font-size:18px;'>❗ Please enter a topic to begin.</span>"),
                                            idx, False, [], True, False ) # empty flashcard state
                                cards = generate_flashcards_from_rag(topic, rag_index, rag_chunks)
                                #cards = generate_flashcards_from_rag(topic)
                                idx = (idx + 1) % len(cards)
                                card = cards[idx]
                                buttons = [("✅ Flip Card 🧐 ", True), ("✅ Flip Card 🤓", False)]
                                random.shuffle(buttons)
                                return (
                                    style_card(f"**Q:** {card['front']}"),
                                    "",
                                    gr.update(visible=True, value=buttons[0][0]),
                                    gr.update(visible=True, value=buttons[1][0]),
                                    gr.update(visible=True),
                                    gr.update(visible=False, value=""),
                                    gr.update(visible=True, value="<span style='color:green; font-size:18px;'>👆 Choose a card to flip!</span>"),
                                    idx, False, cards, buttons[0][1], buttons[1][1]
                                )
                
                            def flip_card(idx, cards, correct):
                                card = cards[idx % len(cards)]
                                front_html = style_card(f"**Q:** {card['front']}")
    
                                if correct:
                                    back_html = style_card(f"✅ Answer: {card['back']}")
                                else:
                                    back_html = gr.update(visible=False)  # hide back card for wrong pick
                                wrong_msgs = [
                                    "🚫 Not quite! As they say, 'Data is the new oil'—but you've hit a dry well. Try the other card!",
                                    "💡 'In God we trust, all others bring data.' Sadly, this card didn't bring it. Flip the other one!",
                                    "🧠 Oops! ‘The greatest value of a picture is when it forces us to notice what we never expected to see.’ You missed it—check the other card.",
                                    "⚙️ 'GPU acceleration turns hours into seconds' — but this pick cost you a moment. Try the other one!",
                                    "📊 'Torture the data long enough, and it will confess to anything.' This card stayed silent. Flip the other!",
                                    "❌ Wrong Pick! 'Without data, you're just another person with an opinion.' Try the other card for some real answers!",
                                    "🚀 'GPUs don’t guess—they compute at scale.' Your guess here missed. Go try the other card!",
                                    "🔄 Oops! You’ve hit a cold cache. Try the other card for a GPU-hot answer.",
                                    "💭 'Data science is the art of turning data into insight'—but this card had none. Flip the other one!"
                                ]
                                msg = "✅ Nailed it! Great job." if correct else random.choice(wrong_msgs)
                            
                                return (
                                    front_html,
                                    back_html,
                                    gr.update(visible=True),
                                    gr.update(visible=True, value=f"<span style='font-size:18px; color:#b00020;'>{msg}</span>"),      # ✅ THIS is the fix
                                    gr.update(visible=False),                # hide instruction
                                    idx,
                                    True,
                                    cards
                                )            
                            btn1_correct = gr.State(True)
                            btn2_correct = gr.State(False)
                
                            gen_btn.click(
                                flashcard_from_query,
                                inputs=[gen_query, flash_idx],
                                outputs=[card_front, card_back, flip_btn1, flip_btn2, reset_btn, feedback_msg, instruction_msg, flash_idx, show_answer, flashcards_state, btn1_correct, btn2_correct]
                            )
                
                            flip_btn1.click(
                                flip_card,
                                inputs=[flash_idx, flashcards_state, btn1_correct],
                                outputs=[card_front, card_back, card_back, feedback_msg, instruction_msg, flash_idx, show_answer, flashcards_state]
                            )
                
                            flip_btn2.click(
                                flip_card,
                                inputs=[flash_idx, flashcards_state, btn2_correct],
                                outputs=[card_front, card_back, card_back, feedback_msg, instruction_msg, flash_idx, show_answer, flashcards_state]
                            )
                
                            reset_btn.click(
                                flashcard_from_query,
                                inputs=[gen_query, flash_idx],
                                outputs=[card_front, card_back, flip_btn1, flip_btn2, reset_btn, feedback_msg, instruction_msg, flash_idx, show_answer, flashcards_state, btn1_correct, btn2_correct]
                            )
                        
                    # === QUIZ ===
                    with gr.Tab("❓ Quiz"):
                        quiz_topic = gr.Textbox(label="Quiz Topic", placeholder="e.g., pandas, cuDF, RAPIDS",  elem_classes="textBoxColor")
                        gen_quiz_btn = gr.Button("🧠 Generate Quiz")
                        
                        quiz_q1 = gr.Markdown()
                        quiz_r1 = gr.Radio(choices=[], label="Answer:", visible=False)
                        quiz_q2 = gr.Markdown()
                        quiz_r2 = gr.Radio(choices=[], label="Answer:", visible=False)
                        
                        submit_btn = gr.Button("Submit Quiz", visible=False)
                        quiz_result = gr.Markdown()
                        quiz_state = gr.State([])
                        
                        def generate_quiz_questions(topic):
                            if not topic:
                                return "Enter a topic!", gr.update(visible=False), "", gr.update(visible=False), gr.update(visible=False), []
                            
                            questions = generate_quiz(topic, 2)
                            return (
                                f"**Q1:** {questions[0]['question']}",
                                gr.update(choices=questions[0]['options'], visible=True, value=None),
                                f"**Q2:** {questions[1]['question']}",
                                gr.update(choices=questions[1]['options'], visible=True, value=None),
                                gr.update(visible=True),
                                questions
                            )
                        
                        def evaluate_quiz(ans1, ans2, questions):
                            if not questions:
                                return "Quiz not loaded!", update_progress()
                            
                            correct = 0
                            result_parts = []
                            
                            # Check Q1
                            correct_ans1 = questions[0]['options'][questions[0]['answer_idx']]
                            if ans1 and ans1 == correct_ans1:
                                correct += 1
                                result_parts.append(f"✅ **Q1:** Correct! ({ans1})")
                            else:
                                result_parts.append(f"❌ **Q1:** Wrong. You answered: {ans1 or 'None'}")
                                result_parts.append(f"    💡 **Correct answer:** {correct_ans1}")
                            
                            # Check Q2  
                            correct_ans2 = questions[1]['options'][questions[1]['answer_idx']]
                            if ans2 and ans2 == correct_ans2:
                                correct += 1
                                result_parts.append(f"✅ **Q2:** Correct! ({ans2})")
                            else:
                                result_parts.append(f"❌ **Q2:** Wrong. You answered: {ans2 or 'None'}")
                                result_parts.append(f"    💡 **Correct answer:** {correct_ans2}")
                            
                            user_state.points += correct * 5
                            
                            # Overall result
                            score_emoji = "🎉" if correct == 2 else "👍" if correct == 1 else "📚"
                            result_parts.insert(0, f"{score_emoji} **Final Score: {correct}/2 correct! +{correct*5} XP**")
                            result_parts.append(f"\n🏆 **Total XP:** {user_state.points}")
                            
                            return "\n".join(result_parts), update_progress()
                        
                        gen_quiz_btn.click(generate_quiz_questions, inputs=quiz_topic,
                                         outputs=[quiz_q1, quiz_r1, quiz_q2, quiz_r2, submit_btn, quiz_state])
                        submit_btn.click(evaluate_quiz, inputs=[quiz_r1, quiz_r2, quiz_state], outputs=[quiz_result, progress])
                    
                    # === CODING PUZZLES ===
                    with gr.Tab("💻 Coding"):
                        gr.Markdown("""
### 🚀 **GPU Speedup Challenge**
Convert pandas code to cuDF and witness the power of GPU acceleration!

**🎯 Your Mission:**
1. Choose difficulty level
2. Convert CPU (pandas) code to GPU (cuDF)
3. **See real-time performance benchmarking** with profiling analysis!

**⚡ Expected Performance Gains:**
- **Beginner:** 6x speedup
- **Intermediate:** 6.4x speedup  
- **Advanced:** 7x speedup

**🔬 Enhanced Benchmarking Features:**
- **Real-time execution** with `perf_counter` precision timing
- **Code profiling** with `cProfile` for detailed function analysis
- **Live performance comparison** between your code and reference
- **Educational insights** about when GPU acceleration helps vs. hurts
""")
                        difficulty = gr.Radio(choices=["Beginner", "Intermediate", "Advanced"], 
                                            value="Beginner", label="Difficulty")
                        gen_puzzle_btn = gr.Button("🎯 Generate Puzzle")
                        
                        puzzle_desc = gr.Markdown()
                        cpu_code = gr.Code(label="CPU Code (pandas)", interactive=False)
                        user_code = gr.Code(label="Your GPU Code (cuDF)", language="python")
                        
                        with gr.Row():
                            check_btn = gr.Button("✅ Check Solution", variant="secondary")
                            benchmark_btn = gr.Button("⚡ LIVE BENCHMARK 🔬", variant="primary", size="lg", elem_classes="benchmark-btn")
                        
                        gr.Markdown("### 📊 **Performance Results:**")
                        puzzle_feedback = gr.Markdown()
                        puzzle_state = gr.State({})
                        
                        def generate_puzzle(diff):
                            puzzle = generate_coding_puzzle(diff)
                            expected_speedup = round(puzzle['cpu_time'] / puzzle['gpu_time'], 1)
                            
                            desc = f"""**🎯 {diff} Challenge:** {puzzle['description']}

🚀 **Expected Speedup:** {expected_speedup}x faster with GPU acceleration!
⏱️ **Performance Target:** {puzzle['cpu_time']}s → {puzzle['gpu_time']}s

💡 **Goal:** Convert the pandas code below to cuDF and unlock GPU performance!"""
                            return desc, puzzle['cpu_code'], "", puzzle
                        
                        def check_solution(user_code_input, puzzle):
                            if not puzzle or not user_code_input:
                                return "Generate a puzzle and enter your solution!", update_progress()
                            
                            # Comprehensive code validation
                            def validate_cudf_solution(user_code, expected_code):
                                user_code = user_code.strip()
                                expected_code = expected_code.strip()
                                
                                # Check 1: Basic syntax validation
                                try:
                                    compile(user_code, '<string>', 'exec')
                                except SyntaxError as e:
                                    return False, f"❌ **Syntax Error:** {str(e)}\n\n💡 Check your Python syntax!"
                                
                                # Check 2: Required imports
                                if 'import cudf' not in user_code and 'from cudf' not in user_code:
                                    return False, "❌ **Missing cuDF import!**\n\n💡 You must import cudf to use GPU acceleration."
                                
                                # Check 3: No pandas usage (common mistake)
                                if 'import pandas' in user_code or 'pd.' in user_code:
                                    return False, "❌ **Still using pandas!**\n\n💡 Replace all pandas/pd references with cudf."
                                
                                # Check 4: Required cuDF operations
                                expected_ops = []
                                if 'cudf.DataFrame' in expected_code:
                                    expected_ops.append('cudf.DataFrame')
                                if '.groupby(' in expected_code:
                                    expected_ops.append('.groupby(')
                                if '.sum()' in expected_code:
                                    expected_ops.append('.sum()')
                                if '.agg(' in expected_code:
                                    expected_ops.append('.agg(')
                                if '.mean()' in expected_code:
                                    expected_ops.append('.mean()')
                                
                                missing_ops = [op for op in expected_ops if op not in user_code]
                                if missing_ops:
                                    return False, f"❌ **Missing operations:** {', '.join(missing_ops)}\n\n💡 Your solution should include all required operations."
                                
                                # Check 5: Similarity check as final validation
                                similarity = difflib.SequenceMatcher(None, user_code, expected_code).ratio()
                                if similarity < 0.8:  # Stricter threshold
                                    return False, f"❌ **Solution not close enough to expected pattern.**\n\n💡 **Your similarity:** {similarity:.2f} (need ≥0.80)\n\n**Expected pattern:**\n```python\n{expected_code}\n```"
                                
                                return True, f"✅ **Valid cuDF solution!** (similarity: {similarity:.2f})"
                            
                            # Validate the user's solution
                            is_valid, feedback = validate_cudf_solution(user_code_input, puzzle['gpu_code'])
                            
                            if is_valid:
                                # Only award points for truly valid solutions
                                similarity = difflib.SequenceMatcher(None, 
                                                                   user_code_input.strip(), 
                                                                   puzzle['gpu_code'].strip()).ratio()
                                
                                if similarity > 0.95:
                                    user_state.points += 10
                                    return f"🎉 **Perfect solution!** +10 XP | Total: {user_state.points} XP\n\n{feedback}", update_progress()
                                elif similarity > 0.85:
                                    user_state.points += 8
                                    return f"🌟 **Excellent solution!** +8 XP | Total: {user_state.points} XP\n\n{feedback}", update_progress()
                                elif similarity > 0.80:
                                    user_state.points += 5
                                    return f"✅ **Good solution!** +5 XP | Total: {user_state.points} XP\n\n{feedback}", update_progress()
                                else:
                                    # This should rarely happen due to validation above
                                    user_state.points += 3
                                    return f"👍 **Acceptable solution!** +3 XP | Total: {user_state.points} XP\n\n{feedback}", update_progress()
                            else:
                                # No points for invalid solutions
                                return f"{feedback}\n\n**🔍 Debug your code step by step:**\n1. Check imports (use `import cudf`)\n2. Replace `pd.` with `cudf.`\n3. Ensure all operations match the expected pattern", update_progress()
                        
                        def run_benchmark(user_code_input, puzzle):
                            if not puzzle:
                                return "Generate a puzzle first!"
                            
                            # Import required timing and profiling modules
                            import time
                            import io
                            import sys
                            import cProfile
                            import pstats
                            from contextlib import redirect_stdout, redirect_stderr
                            
                            # Use the same validation logic as check_solution
                            def validate_cudf_solution(user_code, expected_code):
                                user_code = user_code.strip()
                                expected_code = expected_code.strip()
                                
                                # Check 1: Basic syntax validation
                                try:
                                    compile(user_code, '<string>', 'exec')
                                except SyntaxError:
                                    return False
                                
                                # Check 2: Required imports
                                if 'import cudf' not in user_code and 'from cudf' not in user_code:
                                    return False
                                
                                # Check 3: No pandas usage (common mistake)
                                if 'import pandas' in user_code or 'pd.' in user_code:
                                    return False
                                
                                # Check 4: Required cuDF operations
                                expected_ops = []
                                if 'cudf.DataFrame' in expected_code:
                                    expected_ops.append('cudf.DataFrame')
                                if '.groupby(' in expected_code:
                                    expected_ops.append('.groupby(')
                                if '.sum()' in expected_code:
                                    expected_ops.append('.sum()')
                                if '.agg(' in expected_code:
                                    expected_ops.append('.agg(')
                                if '.mean()' in expected_code:
                                    expected_ops.append('.mean()')
                                
                                missing_ops = [op for op in expected_ops if op not in user_code]
                                if missing_ops:
                                    return False
                                
                                # Check 5: Similarity check as final validation
                                similarity = difflib.SequenceMatcher(None, user_code, expected_code).ratio()
                                return similarity >= 0.8
                            
                            # Validate before showing benchmark
                            is_valid = validate_cudf_solution(user_code_input, puzzle['gpu_code'])
                            
                            if not is_valid:
                                return "❌ **Fix your solution first before benchmarking!**\n\n💡 Your code must:\n• Have valid Python syntax\n• Import cudf (not pandas)\n• Use cuDF operations correctly\n• Match the expected pattern closely\n\n**Click 'Check Solution' first to get detailed feedback.**"
                            
                            def execute_and_time(code_str, description, is_gpu_code=False):
                                """Execute code and return timing results with profiling and device handling"""
                                try:
                                    # Create a clean namespace for execution
                                    namespace = {}
                                    
                                    # Handle device-specific code execution
                                    if is_gpu_code and not device_info['cudf_available']:
                                        # Convert cuDF code to pandas for fallback
                                        fallback_code = code_str.replace('import cudf', 'import pandas as pd')
                                        fallback_code = fallback_code.replace('cudf.', 'pd.')
                                        print(f"⚠️ cuDF not available - using pandas fallback for {description}")
                                        code_to_execute = fallback_code
                                        actual_description = f"{description} (pandas fallback)"
                                    else:
                                        code_to_execute = code_str
                                        actual_description = description
                                    
                                    # Add device information to namespace for debugging
                                    namespace['__device_info__'] = device_info
                                    
                                    # Redirect stdout to capture print statements
                                    captured_output = io.StringIO()
                                    
                                    # Timing with perf_counter for high precision
                                    start_time = time.perf_counter()
                                    
                                    # Profile the execution
                                    profiler = cProfile.Profile()
                                    profiler.enable()
                                    
                                    # Execute the code with output capture
                                    with redirect_stdout(captured_output), redirect_stderr(captured_output):
                                        exec(code_to_execute, namespace)
                                    
                                    profiler.disable()
                                    end_time = time.perf_counter()
                                    
                                    # Calculate execution time
                                    execution_time = end_time - start_time
                                    
                                    # Get profiling stats
                                    stats_stream = io.StringIO()
                                    stats = pstats.Stats(profiler, stream=stats_stream)
                                    stats.sort_stats('cumulative').print_stats(5)  # Top 5 functions
                                    
                                    # Add device information to the result
                                    return {
                                        'time': execution_time,
                                        'output': captured_output.getvalue(),
                                        'profiling': stats_stream.getvalue(),
                                        'success': True,
                                        'description': actual_description,
                                        'using_fallback': is_gpu_code and not device_info['cudf_available'],
                                        'device': device_info['cudf_device'] if is_gpu_code else 'CPU'
                                    }
                                    
                                except Exception as e:
                                    error_msg = str(e)
                                    # Provide specific help for cuDF-related errors
                                    if 'cudf' in error_msg.lower() and not device_info['cudf_available']:
                                        error_msg += "\n💡 Hint: cuDF requires GPU and proper installation. Using pandas fallback."
                                    
                                    return {
                                        'time': float('inf'),
                                        'output': f"Error: {error_msg}",
                                        'profiling': "",
                                        'success': False,
                                        'description': description,
                                        'using_fallback': is_gpu_code and not device_info['cudf_available'],
                                        'device': 'Error'
                                    }
                            
                            # Run benchmarks for both CPU and GPU code
                            print("🔥 Running real-time benchmarks...")
                            
                            # Benchmark CPU code (pandas)
                            cpu_result = execute_and_time(puzzle['cpu_code'], "CPU (pandas)", is_gpu_code=False)
                            
                            # Benchmark GPU code (user's cuDF solution)
                            gpu_result = execute_and_time(user_code_input.strip(), "GPU (cuDF)", is_gpu_code=True)
                            
                            # Check if both executions were successful
                            if not cpu_result['success']:
                                return f"❌ **CPU code execution failed:** {cpu_result['output']}"
                            
                            if not gpu_result['success']:
                                return f"❌ **Your GPU code execution failed:** {gpu_result['output']}\n\n💡 Debug your code and try again."
                            
                            # Calculate actual speedup
                            if gpu_result['time'] > 0:
                                actual_speedup = cpu_result['time'] / gpu_result['time']
                            else:
                                actual_speedup = float('inf')
                            
                            # Get similarity score
                            similarity = difflib.SequenceMatcher(None, 
                                                               user_code_input.strip(), 
                                                               puzzle['gpu_code'].strip()).ratio()
                            
                            # Use actual timing data
                            cpu_time = cpu_result['time']
                            gpu_time = gpu_result['time']
                            speedup = round(actual_speedup, 2)
                            
                            # Create visual speedup comparison
                            cpu_bar_width = 100
                            gpu_bar_width = max(10, int(100 / max(speedup, 0.1)))
                            
                            # Generate ASCII chart with real timings
                            ascii_chart = create_speedup_visualization(cpu_time, gpu_time, speedup)
                            
                            # Determine speedup performance level
                            if speedup >= 5:
                                performance_emoji = "🔥"
                                performance_text = "INCREDIBLE"
                                color = "#ff6b35"
                            elif speedup >= 3:
                                performance_emoji = "⚡"
                                performance_text = "EXCELLENT"
                                color = "#f7931e"
                            elif speedup >= 2:
                                performance_emoji = "🚀"
                                performance_text = "GREAT"
                                color = "#ffd23f"
                            elif speedup >= 1:
                                performance_emoji = "📈"
                                performance_text = "GOOD"
                                color = "#06d6a0"
                            else:
                                performance_emoji = "⚠️"
                                performance_text = "SLOWER"
                                color = "#e74c3c"
                            
                            # Format timing data for display
                            def format_time(t):
                                if t < 1e-6:
                                    return f"{t*1e9:.2f}ns"
                                elif t < 1e-3:
                                    return f"{t*1e6:.2f}μs"
                                elif t < 1:
                                    return f"{t*1e3:.2f}ms"
                                else:
                                    return f"{t:.4f}s"
                            
                            # Extract key profiling information
                            cpu_profile_lines = cpu_result['profiling'].split('\n')[:10]  # First 10 lines
                            gpu_profile_lines = gpu_result['profiling'].split('\n')[:10]  # First 10 lines
                            
                            # Determine device status and create appropriate messaging
                            gpu_device_status = "🚀 GPU (cuDF)" if not gpu_result.get('using_fallback', False) else "⚠️ GPU (pandas fallback)"
                            device_warning = ""
                            
                            if gpu_result.get('using_fallback', False):
                                device_warning = f"""
### ⚠️ **DEVICE STATUS NOTICE:**
**cuDF Availability:** {device_info['cudf_available']} ({'Available' if device_info['cudf_available'] else 'Not Available'})
**GPU Status:** {device_info['has_gpu']} ({'Available' if device_info['has_gpu'] else 'Not Available'})
**Fallback Mode:** Using pandas simulation for GPU code
**Impact:** Results may not reflect true GPU acceleration performance

---
"""
                            
                            benchmark_html = f"""
# 🏁 **REAL-TIME BENCHMARK RESULTS**

## 🔧 **SYSTEM CONFIGURATION:**
- **Main Device:** {device_info['device_name']}
- **GPU Available:** {'✅ Yes' if device_info['has_gpu'] else '❌ No'}
- **cuDF Status:** {'✅ Available' if device_info['cudf_available'] else '❌ Not Available'}
- **Execution Mode:** {'Real GPU acceleration' if device_info['cudf_available'] else 'Pandas simulation'}

{device_warning}

## ⚡ **LIVE PERFORMANCE ANALYSIS**
### {performance_emoji} **{speedup}x {'FASTER' if speedup >= 1 else 'SLOWER'}** - {performance_text} PERFORMANCE!

{ascii_chart}

---

### ⏱️ **Precise Timing Results (perf_counter):**

**🖥️ CPU (pandas) Execution:**
- **Device:** {cpu_result.get('device', 'CPU')}
- **Time:** `{format_time(cpu_time)}`
- **Output:** `{cpu_result['output'][:100]}{'...' if len(cpu_result['output']) > 100 else ''}`

**{gpu_device_status} Execution:**
- **Device:** {gpu_result.get('device', 'Unknown')}
- **Time:** `{format_time(gpu_time)}`
- **Output:** `{gpu_result['output'][:100]}{'...' if len(gpu_result['output']) > 100 else ''}`
- **Using Fallback:** {'Yes (pandas simulation)' if gpu_result.get('using_fallback', False) else 'No (real cuDF)'}

---

### 📊 **Visual Performance Comparison:**

**🖥️ CPU (pandas):**
<div style='background: linear-gradient(90deg, #e74c3c, #c0392b); height: 25px; width: {cpu_bar_width}%; border-radius: 12px; color: white; text-align: center; line-height: 25px; font-weight: bold; margin: 5px 0; box-shadow: 0 2px 4px rgba(0,0,0,0.2);'>
{format_time(cpu_time)}
</div>

**{gpu_device_status}:**
<div style='background: linear-gradient(90deg, {"#f39c12, #e67e22" if gpu_result.get("using_fallback", False) else "#27ae60, #2ecc71"}); height: 25px; width: {gpu_bar_width}%; border-radius: 12px; color: white; text-align: center; line-height: 25px; font-weight: bold; margin: 5px 0; box-shadow: 0 2px 4px rgba(0,0,0,0.2);'>
{format_time(gpu_time)}
</div>

---

### 🔬 **Profiling Analysis (Top Functions):**

**CPU Profiling:**
```
{''.join(cpu_profile_lines)}
```

**GPU Profiling:**
```
{''.join(gpu_profile_lines)}
```

---

### 🎯 **Detailed Metrics:**
- **⏱️ Absolute Time Difference:** {format_time(abs(cpu_time - gpu_time))}
- **📊 Percentage Improvement:** {((speedup - 1) * 100):.1f}% {'faster' if speedup >= 1 else 'slower'}
- **🎯 Code Similarity:** {similarity:.1%} match with expected solution
- **💰 Performance Category:** {performance_text}
- **🔋 Execution Efficiency:** {'GPU parallel processing' if speedup >= 1 else 'CPU sequential processing'} performed better

### 📈 **Real-World Implications:**
- **Current Dataset:** {'GPU' if not gpu_result.get('using_fallback', False) else 'Simulated GPU'} solution is **{speedup:.2f}x {'faster' if speedup >= 1 else 'slower'}**
- **Large Dataset (1M+ rows):** {'Could be 10-100x faster with real GPU!' if gpu_result.get('using_fallback', False) else 'Could be 10-100x faster!' if speedup >= 1 else 'Consider pandas for this workload size'}
- **Production Scale:** {'Install cuDF and use GPU for massive savings 💰' if gpu_result.get('using_fallback', False) else 'Massive time and cost savings 💰' if speedup >= 1 else 'Pandas might be more suitable 🤔'}

### 🚀 **Technical Insights:**
- **Parallel Processing:** {'GPU acceleration simulated - real GPU would utilize thousands of cores' if gpu_result.get('using_fallback', False) else 'GPU utilizes thousands of cores effectively' if speedup >= 1 else 'CPU single-threaded execution was sufficient'}
- **Memory Bandwidth:** {'Real GPU would provide massive memory throughput advantage' if gpu_result.get('using_fallback', False) else 'GPU memory throughput advantage' if speedup >= 1 else 'CPU memory access was adequate'}
- **Overhead Analysis:** {'cuDF setup overhead simulated - real GPU overhead often justified for larger datasets' if gpu_result.get('using_fallback', False) else 'cuDF initialization overhead was justified' if speedup >= 1 else 'cuDF overhead exceeded benefits for this dataset size'}

{performance_emoji} **{'Set up cuDF and GPU to see real acceleration!' if gpu_result.get('using_fallback', False) else 'Congratulations on mastering GPU acceleration!' if speedup >= 1 else 'Keep practicing - pandas was better for this case!'}** {performance_emoji}

---

### 🧠 **Learning Notes:**
- **Small datasets** (< 10K rows): Pandas often faster due to lower overhead
- **Medium datasets** (10K - 1M rows): cuDF starts showing benefits{'📝 Note: Results simulated - real GPU would show even better performance' if gpu_result.get('using_fallback', False) else ''}
- **Large datasets** (> 1M rows): cuDF typically dominates{'🚀 Real GPU acceleration would be dramatic here!' if gpu_result.get('using_fallback', False) else ''}
- **Your result:** {performance_text.lower()} performance indicates {'simulation mode - real GPU would likely perform much better' if gpu_result.get('using_fallback', False) else 'optimal GPU utilization' if speedup >= 2 else 'room for optimization' if speedup >= 1 else 'pandas was the right choice here'}

### 🔧 **Setup Recommendations:**
{
f'''
- **Install cuDF:** `conda install -c rapidsai -c conda-forge -c nvidia cudf python=3.10 cudatoolkit=11.8`
- **GPU Requirements:** NVIDIA GPU with CUDA support
- **Memory:** At least 8GB GPU memory recommended
- **Alternative:** Use Google Colab with GPU runtime for testing
''' if gpu_result.get('using_fallback', False) else '''
- **Optimization:** Your GPU setup is working perfectly!
- **Next Steps:** Try larger datasets to see dramatic speedups
- **Advanced:** Explore cuML for machine learning acceleration
'''
}
"""
                            return benchmark_html
                        
                        gen_puzzle_btn.click(generate_puzzle, inputs=difficulty, 
                                           outputs=[puzzle_desc, cpu_code, user_code, puzzle_state])
                        check_btn.click(check_solution, inputs=[user_code, puzzle_state], outputs=[puzzle_feedback, progress])
                        benchmark_btn.click(run_benchmark, inputs=[user_code, puzzle_state], outputs=puzzle_feedback)
                
                # Update progress on point changes
                def refresh_progress():
                    return update_progress()
        
        # Footer
        gr.HTML("""
        ---
        <div style='text-align:center; color:; padding:15px;'>
        🌟 • Structured Learning Paths • Socratic Guidance • Gamified Practice<br>
        📚 <i>From beginner tutorials to advanced GPU computing - your complete data science learning companion!</i><br>
        </div>
        """)
    
    return app

# === LAUNCH ===
if __name__ == "__main__":
    print("\n🚀 Creating AI Tutor interface...")
    app = create_ai_tutor()
    
    print("🎉 AI Tutor ready!")
    print("=" * 60)
    print("✅ Complete Learning System Available:")
    print("  📚 Learning Mode - Structured topic-based learning paths")
    print("  🎓 Tutor Mode - Socratic Q&A with guided discovery")
    print("  🎮 Game Mode - XP-based practice with flashcards, quizzes & coding")
    print("  🌟 Dynamic content generation with contextual quotes")
    print("  📖 Enhanced documentation (12+ sources)")
    print("  🎯 From beginner Python to advanced GPU computing")
    print("=" * 60)

    # Queue requests
    app.queue(max_size=15, api_open=True)
    # Launch the app
    app.launch(share=True, max_threads=10)

🚀 GPU DETECTED: NVIDIA A100-SXM4-80GB
📊 GPU Memory: 79.3 GB
🔢 GPU Count: 1
✅ GPU is available and functional
🔧 Loading SentenceTransformer on CUDA...
✅ SentenceTransformer loaded successfully on CUDA
🤖 Loading NousResearch/Hermes-2-Pro-Mistral-7B
🚀 Loading model on GPU: NVIDIA A100-SXM4-80GB


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda


✅ Model loaded successfully on GPU
⚠️ cuDF not installed - will use pandas fallback in benchmarks

🔧 **DEVICE CONFIGURATION SUMMARY:**
📱 Main Device: CUDA
🚀 GPU Available: ✅ Yes
🖥️ Device Name: NVIDIA A100-SXM4-80GB
💾 GPU Memory: 79.3 GB
📊 cuDF Status: ⚠️ Using pandas fallback
🎯 Benchmarking Mode: Simulated with pandas
--------------------------------------------------
✅ LLM loaded and ready!
🚀 Initializing RAG system...
🔄 Initializing RAG system...
✅ Loaded cached RAG with 123 chunks
✅ Using cached RAG with 123 chunks

🚀 Creating AI Tutor interface...
🎉 AI Tutor ready!
✅ Complete Learning System Available:
  📚 Learning Mode - Structured topic-based learning paths
  🎓 Tutor Mode - Socratic Q&A with guided discovery
  🎮 Game Mode - XP-based practice with flashcards, quizzes & coding
  🌟 Dynamic content generation with contextual quotes
  📖 Enhanced documentation (12+ sources)
  🎯 From beginner Python to advanced GPU computing
* Running on local URL:  http://127.0.0.1:7861
* Running on p